In [1]:
import trimesh
import numpy as np
import pymesh
import psutil
import reutils as utils
import time
import time
import relegolization1 as lego
#import reutils as utils
#import relegolization1 as lego
import fabrication as fab
import math
import networkx as nx
import debug_utils as db
from pathlib import Path
import random
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib widget
from scipy import stats
from ast import literal_eval
import seaborn as sns

In [2]:
modelsdf = pd.read_csv("../out/models.csv")
resultsdf = pd.read_csv("../out/results.csv")
componentsdf = pd.read_csv("../out/components.csv")
bestsdf = pd.read_csv("../out/bests.csv")
bestcomponentsdf = pd.read_csv("../out/bestcomponents.csv")


bestsdf = bestsdf.apply(pd.to_numeric, errors='ignore')
bestsdf.bricks = bestsdf.bricks.apply(literal_eval)
bestsdf.times = bestsdf.times.apply(literal_eval)
bestsdf.connected_components = bestsdf.connected_components.apply(literal_eval)
bestsdf.node_connectivity = bestsdf.node_connectivity.apply(literal_eval)
bestsdf.imp_times = bestsdf.imp_times.apply(literal_eval)
bestsdf.imp_iterations = bestsdf.imp_iterations.apply(literal_eval)
bestsdf.imp_bricks = bestsdf.imp_bricks.apply(literal_eval)
bestsdf.imp_connected_components = bestsdf.imp_connected_components.apply(literal_eval)
bestsdf.imp_node_connectivity = bestsdf.imp_node_connectivity.apply(literal_eval)

bestcomponentsdf = bestcomponentsdf.apply(pd.to_numeric, errors='ignore')
bestcomponentsdf.bricks_conn_comps = bestcomponentsdf.bricks_conn_comps.apply(literal_eval)
bestcomponentsdf.voxels_conn_comps = bestcomponentsdf.voxels_conn_comps.apply(literal_eval)
bestcomponentsdf.component_outside = bestcomponentsdf.component_outside.apply(literal_eval)
bestcomponentsdf.imp_bricks_conn_comps = bestcomponentsdf.imp_bricks_conn_comps.apply(literal_eval)
bestcomponentsdf.imp_voxels_conn_comps = bestcomponentsdf.imp_voxels_conn_comps.apply(literal_eval)
bestcomponentsdf.imp_component_outside = bestcomponentsdf.imp_component_outside.apply(literal_eval)

In [45]:
df = modelsdf
low_v = df.loc[ df['resolution'] == 'low']['voxels'].mean()
med_v = df.loc[ df['resolution'] == 'medium']['voxels'].mean()
high_v = df.loc[ df['resolution'] == 'high']['voxels'].mean()
print(low_v,med_v,high_v)

1913.7878787878788 14798.39393939394 129192.09090909091


In [82]:
avg_random_time = resultsdf.loc[ resultsdf['execution_type'] == 'random']['time_avg'].mean()
avg_heur_time = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['time_avg'].mean()
print("Random average execution time",avg_random_time)
print("Heuristic average execution time",avg_heur_time)
print("Improv of averages",abs(avg_random_time - avg_heur_time)/avg_random_time)
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'time_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'time_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['time_avg_r'] - diff['time_avg']
diff['perc'] = (diff['time_avg_r'] - diff['time_avg'])/diff['time_avg_r']
diff['perc'].var() # ((diff15['perc'] - diff15['perc'].mean())**2).sum()/(diff15.shape[0]-1)
t = (diff['perc'].mean() - 0)/((diff['perc'].std())/(np.sqrt(diff.shape[0])))
print("Averages of perc imvprov", diff['perc'].mean())
print("t", t)
#
avg_random_time = resultsdf.loc[ resultsdf['execution_type'] == 'random']['time_avg']
avg_heur_time = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['time_avg']
print(stats.ttest_rel(avg_random_time,avg_heur_time))


Random average execution time 48.93835907343665
Heuristic average execution time 39.76554968621997
Improv of averages 0.1874359819349891
Averages of perc imvprov 0.15561067423518138
t 39.17599115352312
Ttest_relResult(statistic=7.075087576659351, pvalue=2.247248195998038e-10)


In [141]:
from scipy.stats import norm
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'time_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'time_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['time_avg_r'] - diff['time_avg']
diff['perc'] = (diff['time_avg_r'] - diff['time_avg'])/diff['time_avg_r']
diff['perc'].hist()
# x axis for normal plot
x_axis = np.arange(diff['perc'].min(), diff['perc'].max(), 0.001)
plt.figure()
diff['perc'].plot.hist()

#plt.plot(x_axis, norm.pdf(x_axis,diff['perc'].mean(),diff['perc'].std()))
plt.xlabel("Percentage of time improvement")
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
diff['perc'].std()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.03952182481789545

In [256]:
# Plot time over voxels in random v euristic
df = resultsdf.merge(modelsdf.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
x_axis_r = df.loc[(df['execution_type'] == 'random' )]['voxels']
y_axis_r = df.loc[(df['execution_type'] == 'random' )]['time_avg']

x_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['voxels']
y_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['time_avg']

plt.figure()
plt.plot(x_axis_r, y_axis_r, 'ob')
plt.plot(x_axis_e, y_axis_e, 'or')
plt.xlabel('voxels')
plt.ylabel('seconds')
plt.yscale('log')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
avg_random_time = resultsdf.loc[(resultsdf['execution_type'] == 'random') & (resultsdf['resolution'] == 'low') ]['time_avg'].mean()
avg_heur_time = resultsdf.loc[(resultsdf['execution_type'] == 'euristic') & (resultsdf['resolution'] == 'low')]['time_avg'].mean()
print("Random average execution time",avg_random_time)
print("Heuristic average execution time",avg_heur_time)
avg_random_time = resultsdf.loc[(resultsdf['execution_type'] == 'random') & (resultsdf['resolution'] == 'medium') ]['time_avg'].mean()
avg_heur_time = resultsdf.loc[(resultsdf['execution_type'] == 'euristic') & (resultsdf['resolution'] == 'medium')]['time_avg'].mean()
print("Random average execution time",avg_random_time)
print("Heuristic average execution time",avg_heur_time)
avg_random_time = resultsdf.loc[(resultsdf['execution_type'] == 'random') & (resultsdf['resolution'] == 'high') ]['time_avg'].mean()
avg_heur_time = resultsdf.loc[(resultsdf['execution_type'] == 'euristic') & (resultsdf['resolution'] == 'high')]['time_avg'].mean()
print("Random average execution time",avg_random_time)
print("Heuristic average execution time",avg_heur_time)

Random average execution time 0.44749489553046945
Heuristic average execution time 0.3894680839596373
Random average execution time 4.756530727762164
Heuristic average execution time 4.081986474268364
Random average execution time 141.61105159701725
Heuristic average execution time 114.8251945004319


In [39]:
avg_random_iterations = resultsdf.loc[ resultsdf['execution_type'] == 'random']['imp_iterations_avg'].mean()
avg_heur_iterations = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['imp_iterations_avg'].mean()
print("Random average execution iterations",avg_random_iterations)
print("Heuristic average execution iterations",avg_heur_iterations)
print("Improv of averages",abs(avg_random_iterations - avg_heur_iterations)/avg_random_iterations)
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'imp_iterations_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'imp_iterations_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['imp_iterations_avg_r'] - diff['imp_iterations_avg']
diff['perc'] = (diff['imp_iterations_avg_r'] - diff['imp_iterations_avg'])/diff['imp_iterations_avg_r']
diff['perc'].var() # ((diff15['perc'] - diff15['perc'].mean())**2).sum()/(diff15.shape[0]-1)
t = (diff['perc'].mean() - 0)/((diff['perc'].var())/(np.sqrt(diff.shape[0])))
print("Averages of perc imvprov", diff['perc'].mean())
print("t", t)

Random average execution iterations 6.754545454545456
Heuristic average execution iterations 1.5070707070707063
Improv of averages 0.7768805144309856
Averages of perc imvprov 0.8015168132163875
t 84.72578558933063


In [38]:
avg_random_time = resultsdf.loc[ resultsdf['execution_type'] == 'random']['imp_time_avg'].mean()
avg_heur_time = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['imp_time_avg'].mean()
print("Random average execution time",avg_random_time)
print("Heuristic average execution time",avg_heur_time)
print("Improv of averages",abs(avg_random_time - avg_heur_time)/avg_random_time)
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'imp_time_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'imp_time_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['imp_time_avg_r'] - diff['imp_time_avg']
diff['perc'] = (diff['imp_time_avg_r'] - diff['imp_time_avg'])/diff['imp_time_avg_r']
diff['perc'].var() # ((diff15['perc'] - diff15['perc'].mean())**2).sum()/(diff15.shape[0]-1)
t = (diff['perc'].mean() - 0)/((diff['perc'].var())/(np.sqrt(diff.shape[0])))
print("Averages of perc imvprov", diff['perc'].mean())
print("t", t)

Random average execution time 95.85888830989298
Heuristic average execution time 10.222492337226866
Improv of averages 0.8933589517105649
Averages of perc imvprov 0.8354576459481216
t 153.47638645568807


In [32]:
avg_random_comps = resultsdf.loc[ resultsdf['execution_type'] == 'random']['connected_components_avg'].mean()
avg_heur_comps = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['connected_components_avg'].mean()
print("Random average execution components",avg_random_comps)
print("Heuristic average execution components",avg_heur_comps)
print("Improv of averages",abs(avg_random_comps - avg_heur_comps)/avg_random_comps)
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'connected_components_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'connected_components_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['connected_components_avg_r'] - diff['connected_components_avg']
diff['perc'] = (diff['connected_components_avg_r'] - diff['connected_components_avg'])/diff['connected_components_avg_r']
diff['perc'].var() # ((diff15['perc'] - diff15['perc'].mean())**2).sum()/(diff15.shape[0]-1)
t = (diff['perc'].mean() - 0)/((diff['perc'].var())/(np.sqrt(diff.shape[0])))
print("Averages of perc imvprov", diff['perc'].mean())
print("t", t,diff['perc'].var(),diff['perc'].std(), np.sqrt(diff['perc'].var()), ((diff['perc'] - diff['perc'].mean())**2).sum()/(diff.shape[0]-1))

Random average execution components 7.662626262626262
Heuristic average execution components 1.7404040404040406
Improv of averages 0.7728710783021354
Averages of perc imvprov 0.6400901608592916
t 124.61438255588446 0.051108199198831264 0.22607122594180637 0.22607122594180637 0.05059195476247942


In [40]:
avg_random_bricks = resultsdf.loc[ resultsdf['execution_type'] == 'random']['bricks_avg'].mean()
avg_heur_bricks = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['bricks_avg'].mean()
print("Random average execution bricks",avg_random_bricks)
print("Heuristic average execution bricks",avg_heur_bricks)
print("Improv of averages",abs(avg_random_bricks - avg_heur_bricks)/avg_random_bricks)
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'bricks_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'bricks_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['bricks_avg_r'] - diff['bricks_avg']
diff['perc'] = (diff['bricks_avg_r'] - diff['bricks_avg'])/diff['bricks_avg_r']
diff['perc'].var() # ((diff15['perc'] - diff15['perc'].mean())**2).sum()/(diff15.shape[0]-1)
t = (diff['perc'].mean() - 0)/((diff['perc'].var())/(np.sqrt(diff.shape[0])))
print("Averages of perc imvprov", diff['perc'].mean())
print("t", t)

Random average execution bricks 12972.452525252525
Heuristic average execution bricks 9484.142424242424
Improv of averages 0.2689013580292287
Averages of perc imvprov 0.23852121518682315
t 904.1448799694093


In [91]:
avg_random_comps = resultsdf.loc[ resultsdf['execution_type'] == 'random']['imp_connected_components_avg'].mean()
avg_heur_comps = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['imp_connected_components_avg'].mean()
print("Random average execution components imp_",avg_random_comps)
print("Heuristic average execution components imp_",avg_heur_comps)
print("Improv of averages",abs(avg_random_comps - avg_heur_comps)/avg_random_comps)
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'imp_connected_components_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'imp_connected_components_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['imp_connected_components_avg_r'] - diff['imp_connected_components_avg']
diff['perc'] = (diff['imp_connected_components_avg_r'] - diff['imp_connected_components_avg'])/diff['imp_connected_components_avg_r']
diff['perc'].var() # ((diff15['perc'] - diff15['perc'].mean())**2).sum()/(diff15.shape[0]-1)
t = (diff['diff'].mean() - 0)/((diff['diff'].std())/(np.sqrt(diff.shape[0])))
print("Averages of perc imvprov", diff['perc'].mean())
#print("t", t,diff['perc'].var(),diff['perc'].std(), np.sqrt(diff['perc'].var()), ((diff['perc'] - diff['perc'].mean())**2).sum()/(diff.shape[0]-1))
avg_random_comps = resultsdf.loc[ resultsdf['execution_type'] == 'random']['imp_connected_components_avg']
avg_heur_comps = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['imp_connected_components_avg']
print(stats.ttest_rel(avg_random_comps, avg_heur_comps))

Random average execution components imp_ 1.0121212121212122
Heuristic average execution components imp_ 1.0292929292929294
Improv of averages 0.016966067864271454
Averages of perc imvprov -0.01827364554637282
Ttest_relResult(statistic=-0.98630451317255, pvalue=0.32641258694820774)


In [43]:
avg_random_comps = resultsdf.loc[ resultsdf['execution_type'] == 'random']['imp_bricks_avg'].mean()
avg_heur_comps = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['imp_bricks_avg'].mean()
print("Random average execution components imp_",avg_random_comps)
print("Heuristic average execution components imp_",avg_heur_comps)
print("Improv of averages",abs(avg_random_comps - avg_heur_comps)/avg_random_comps)
df = resultsdf
dfe = df.loc[(df['execution_type'] == 'euristic' )][['model', 'imp_bricks_avg', 'resolution']]
dfr = df.loc[(df['execution_type'] == 'random' )][['model', 'imp_bricks_avg', 'resolution']]
diff = dfr.merge(dfe.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
diff['diff'] = diff['imp_bricks_avg_r'] - diff['imp_bricks_avg']
diff['perc'] = (diff['imp_bricks_avg_r'] - diff['imp_bricks_avg'])/diff['imp_bricks_avg_r']
diff['perc'].var() # ((diff15['perc'] - diff15['perc'].mean())**2).sum()/(diff15.shape[0]-1)
t = (diff['diff'].mean() - 0)/((diff['diff'].std())/(np.sqrt(diff.shape[0])))
print("Averages of perc imvprov", diff['perc'].mean())
print("t", t,diff['perc'].var(),diff['perc'].std(), np.sqrt(diff['perc'].var()), ((diff['perc'] - diff['perc'].mean())**2).sum()/(diff.shape[0]-1))

Random average execution components imp_ 12496.389898989897
Heuristic average execution components imp_ 9471.48888888889
Improv of averages 0.24206199026692612
Averages of perc imvprov 0.21238114261878216
t 8.28780804670169 0.0027982823843258177 0.05289879378894965 0.05289879378894965 0.0027982823843258173


In [98]:
df = resultsdf
random_data = resultsdf.loc[ resultsdf['execution_type'] == 'random']['node_connectivity_avg']
heur_data = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['node_connectivity_avg']
print("Random average node connectivity",random_data.mean())
print("Heuristic average node connectivity",heur_data.mean())
print(stats.ttest_rel(random_data, heur_data))

Random average node connectivity 3.5569696969696967
Heuristic average node connectivity 3.71010101010101
Ttest_relResult(statistic=-2.728001365896136, pvalue=0.007552280190948177)


In [107]:
df = resultsdf
res = 'high'
random_data = resultsdf.loc[ (resultsdf['execution_type'] == 'random') & (resultsdf['resolution'] == res)]['node_connectivity_avg']
heur_data = resultsdf.loc[ (resultsdf['execution_type'] == 'euristic') & (resultsdf['resolution'] == res)]['node_connectivity_avg']
print("Random average node connectivity",random_data.mean())
print("Heuristic average node connectivity",heur_data.mean())
print(stats.ttest_rel(random_data, heur_data))

Random average node connectivity 4.106060606060606
Heuristic average node connectivity 4.744242424242423
Ttest_relResult(statistic=-8.120130081350235, pvalue=2.8389845417169073e-09)


In [118]:
# Importing libraries
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = resultsdf
res = 'high'
#df = df.loc[]["execution_type", "resolution", "node_connectivity_avg"]
# Performing two-way ANOVA
model = ols('node_connectivity_avg ~ C(execution_type) + C(resolution) + C(execution_type):C(resolution)',
            data=df).fit()
result = sm.stats.anova_lm(model, type=2)
  
# Print the result
print(result)

                                    df     sum_sq    mean_sq           F  \
C(execution_type)                  1.0   1.160735   1.160735    6.019320   
C(resolution)                      2.0  84.326668  42.163334  218.649827   
C(execution_type):C(resolution)    2.0   6.835407   3.417704   17.723463   
Residual                         192.0  37.024315   0.192835         NaN   

                                       PR(>F)  
C(execution_type)                1.504221e-02  
C(resolution)                    3.210692e-50  
C(execution_type):C(resolution)  8.638356e-08  
Residual                                  NaN  


In [119]:
df = resultsdf
random_data = resultsdf.loc[ resultsdf['execution_type'] == 'random']['imp_node_connectivity_avg']
heur_data = resultsdf.loc[ resultsdf['execution_type'] == 'euristic']['imp_node_connectivity_avg']
print("Random average node connectivity",random_data.mean())
print("Heuristic average node connectivity",heur_data.mean())
print(stats.ttest_rel(random_data, heur_data))

Random average node connectivity 3.6181818181818186
Heuristic average node connectivity 3.717676767676766
Ttest_relResult(statistic=-1.7905434696055904, pvalue=0.07645431321725848)


In [123]:
df = resultsdf
res = 'low'
random_data = resultsdf.loc[ (resultsdf['execution_type'] == 'random') & (resultsdf['resolution'] == res)]['imp_node_connectivity_avg']
heur_data = resultsdf.loc[ (resultsdf['execution_type'] == 'euristic') & (resultsdf['resolution'] == res)]['imp_node_connectivity_avg']
print("Random average node connectivity",random_data.mean())
print("Heuristic average node connectivity",heur_data.mean())
print(stats.ttest_rel(random_data, heur_data))

Random average node connectivity 3.026666666666667
Heuristic average node connectivity 2.7103030303030304
Ttest_relResult(statistic=7.483258131097381, pvalue=1.613738040707489e-08)


In [124]:
# Importing libraries
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = resultsdf
res = 'high'
#df = df.loc[]["execution_type", "resolution", "node_connectivity_avg"]
# Performing two-way ANOVA
model = ols('imp_node_connectivity_avg ~ C(execution_type) + C(resolution) + C(execution_type):C(resolution)',
            data=df).fit()
result = sm.stats.anova_lm(model, type=2)
  
# Print the result
print(result)

                                    df     sum_sq    mean_sq           F  \
C(execution_type)                  1.0   0.490013   0.490013    2.580909   
C(resolution)                      2.0  83.638868  41.819434  220.264027   
C(execution_type):C(resolution)    2.0   6.565959   3.282979   17.291536   
Residual                         192.0  36.453212   0.189860         NaN   

                                       PR(>F)  
C(execution_type)                1.098029e-01  
C(resolution)                    1.964527e-50  
C(execution_type):C(resolution)  1.244749e-07  
Residual                                  NaN  


In [125]:
# Importing libraries
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = resultsdf
#df = df.loc[]["execution_type", "resolution", "node_connectivity_avg"]
# Performing two-way ANOVA
model = ols('time_avg ~ C(execution_type) + C(resolution) + C(execution_type):C(resolution)',
            data=df).fit()
result = sm.stats.anova_lm(model, type=2)
  
# Print the result
print(result)

                                    df         sum_sq        mean_sq  \
C(execution_type)                  1.0    4164.951387    4164.951387   
C(resolution)                      2.0  696848.095650  348424.047825   
C(execution_type):C(resolution)    2.0    7681.067151    3840.533576   
Residual                         192.0   26358.757703     137.285196   

                                           F         PR(>F)  
C(execution_type)                  30.337950   1.155251e-07  
C(resolution)                    2537.957894  8.309806e-139  
C(execution_type):C(resolution)    27.974856   2.177768e-11  
Residual                                 NaN            NaN  


In [179]:
df = componentsdf
random_data = df.loc[ df['execution_type'] == 'random']['bricks_conn_comp_avg']
heur_data = df.loc[ df['execution_type'] == 'euristic']['bricks_conn_comp_avg']
print("Random average bricks in conn comp",random_data.mean())
print("Heuristic average bricks in conn comp",heur_data.mean())
print(stats.ttest_rel(random_data, heur_data))

Random average bricks in conn comp 1.5401800808074666
Heuristic average bricks in conn comp 2.2602559169225844
Ttest_relResult(statistic=-1.4982754907117433, pvalue=0.13727597580937762)


In [180]:
df = componentsdf
random_data = df.loc[ df['execution_type'] == 'random']['voxels_conn_comp_avg']
heur_data = df.loc[ df['execution_type'] == 'euristic']['voxels_conn_comp_avg']
print("Random average voxels in conn comp",random_data.mean())
print("Heuristic average voxels in conn comp",heur_data.mean())
print(stats.ttest_rel(random_data, heur_data))

Random average voxels in conn comp 3.1467111287468352
Heuristic average voxels in conn comp 7.89423308173308
Ttest_relResult(statistic=-2.1878291989210545, pvalue=0.03105923862699989)


In [183]:
# Importing libraries
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = componentsdf
# Performing two-way ANOVA
model = ols('voxels_conn_comp_avg ~ C(execution_type) + C(resolution) + C(execution_type):C(resolution)',
            data=df).fit()
result = sm.stats.anova_lm(model, type=2)
  
# Print the result
print(result)

                                    df        sum_sq      mean_sq         F  \
C(execution_type)                  1.0   1115.678752  1115.678752  4.779242   
C(resolution)                      2.0    330.291582   165.145791  0.707436   
C(execution_type):C(resolution)    2.0    729.226674   364.613337  1.561897   
Residual                         192.0  44820.984411   233.442627       NaN   

                                   PR(>F)  
C(execution_type)                0.030014  
C(resolution)                    0.494186  
C(execution_type):C(resolution)  0.212391  
Residual                              NaN  


In [168]:
df = resultsdf
group = df.groupby(["resolution","execution_type"], as_index=False)
table = group.mean()
table = table[["resolution", "execution_type", "time_avg","bricks_avg", "connected_components_avg", "node_connectivity_avg"]]
table.to_csv("../out/table1.csv")

In [55]:
df = resultsdf
group = df.groupby(["resolution","execution_type"], as_index=False)
table = group.mean()
table = table[["resolution", "execution_type", "imp_time_avg","imp_bricks_avg", "imp_connected_components_avg", "imp_node_connectivity_avg"]]
table.to_csv("../out/table2.csv")
table

,resolution,execution_type,imp_time_avg,imp_bricks_avg,imp_connected_components_avg,imp_node_connectivity_avg
0,high,euristic,27.291553,24997.896970,1.036364,4.745758
1,high,random,274.178909,33075.481818,1.036364,4.175152
2,low,euristic,0.276059,444.278788,1.000000,2.710303
3,low,random,0.885320,523.342424,1.000000,3.026667
4,medium,euristic,3.099866,2972.290909,1.051515,3.696970
5,medium,random,12.512436,3890.345455,1.000000,3.652727


In [176]:
df = resultsdf
table = df.loc[(df['execution_type'] == 'euristic' ) & (df['resolution'] == 'low' ) ][[ "model", "time_avg","bricks_avg", "connected_components_avg", "node_connectivity_avg"]]
table.to_csv("../out/table3.csv")

In [193]:
df = componentsdf
group = df.groupby(["resolution","execution_type"], as_index=False)
table = group.mean()
table = table[["resolution", "execution_type", "bricks_conn_comp_avg","voxels_conn_comp_avg", "outside_percentage_avg"]]#, "imp_bricks_conn_comp_avg","imp_voxels_conn_comp_avg", "imp_outside_percentage_avg"]]
table.to_csv("../out/table4.csv")
table

,resolution,execution_type,bricks_conn_comp_avg,voxels_conn_comp_avg,outside_percentage_avg
0,high,euristic,2.541414,9.681364,1.0
1,high,random,1.389724,2.286058,1.0
2,low,euristic,1.225556,3.418737,1.0
3,low,random,1.699202,4.098729,1.0
4,medium,euristic,3.013798,10.582598,1.0
5,medium,random,1.531613,3.055347,1.0


In [274]:
# Plot time over components in random v euristic
df = resultsdf.merge(modelsdf.set_index("model"), on=["model","resolution"], suffixes=('_r',''))
x_axis_r = df.loc[(df['execution_type'] == 'random' )]['voxels']
y_axis_r = df.loc[(df['execution_type'] == 'random' )]['connected_components_avg']

x_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['voxels']
y_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['connected_components_avg']

plt.figure()
plt.plot(x_axis_r, y_axis_r, 'ob')
plt.plot(x_axis_e, y_axis_e, 'or')
plt.xlabel('voxels')
plt.ylabel('connected components')
#plt.yscale('log')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
# Plot components over resolution in random v euristic
df = resultsdf
df = df[['connected_components_avg', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Average of connected components'] = df.connected_components_avg
#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
plt.figure()
sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of connected components', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
#sns.swarmplot(data=df, x='Resolution', y='Average of connected components', split=True, hue='Execution type',)
sns.boxplot(data=df, x='Resolution', y='Average of connected components', hue="Execution type")

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [329]:
# plot time on resolution
df = resultsdf
df = df[['time_avg', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Average of elapsed time'] = df.time_avg
#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
plt.figure()
sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
sns.swarmplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type',)

#plt.yscale('log')
plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [145]:
# plot time on resolution
df = resultsdf
df = df[['time_avg', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Average of elapsed time'] = df.time_avg

df1 = df.loc[df.resolution == 'low']
df2 = df.loc[df.resolution == 'medium']
df3 = df.loc[df.resolution == 'high']

#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
fig, axs = plt.subplots(1,3,constrained_layout=True )
sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
g1 = sns.boxplot(data=df1, x='Resolution', y='Average of elapsed time', hue='Execution type',ax=axs[0])
g2 = sns.boxplot(data=df2, x='Resolution', y='Average of elapsed time', hue='Execution type',ax=axs[1])
g3 = sns.boxplot(data=df3, x='Resolution', y='Average of elapsed time', hue='Execution type',ax=axs[2])
g1.set(xlabel=None)
g3.set(xlabel=None)
g1.set(ylabel="Average of elapsed time in seconds")
g2.set(ylabel=None)
g3.set(ylabel=None)
axs[0].legend([],[], frameon=False)
axs[1].legend([],[], frameon=False)

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [147]:
# Plot bricks over resolution in random v euristic
df = resultsdf
df = df[['bricks_avg', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Average of bricks'] = df.bricks_avg

df1 = df.loc[df.resolution == 'low']
df2 = df.loc[df.resolution == 'medium']
df3 = df.loc[df.resolution == 'high']

#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
fig, axs = plt.subplots(1,3,constrained_layout=True )

sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
g1 = sns.boxplot(data=df1, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[0])
g2 = sns.boxplot(data=df2, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[1])
g3 = sns.boxplot(data=df3, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[2])
g1.set(xlabel=None)
g3.set(xlabel=None)
g1.set(ylabel="Average number of bricks")
g2.set(ylabel=None)
g3.set(ylabel=None)
axs[2].legend([],[], frameon=False)
axs[1].legend([],[], frameon=False)

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [52]:
# plot time on resolution
df = resultsdf
df = df[['imp_time_avg', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Average of elapsed time'] = df.imp_time_avg

df1 = df.loc[df.resolution == 'low']
df2 = df.loc[df.resolution == 'medium']
df3 = df.loc[df.resolution == 'high']

#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
fig, axs = plt.subplots(1,3,constrained_layout=True )
sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
g1 = sns.boxplot(data=df1, x='Resolution', y='Average of elapsed time', hue='Execution type',ax=axs[0])
g2 = sns.boxplot(data=df2, x='Resolution', y='Average of elapsed time', hue='Execution type',ax=axs[1])
g3 = sns.boxplot(data=df3, x='Resolution', y='Average of elapsed time', hue='Execution type',ax=axs[2])
g1.set(xlabel=None)
g3.set(xlabel=None)
g1.set(ylabel="Average of optimization elapsed time in seconds")
g2.set(ylabel=None)
g3.set(ylabel=None)
axs[2].legend([],[], frameon=False)
axs[1].legend([],[], frameon=False)

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [54]:
# Plot bricks over resolution in random v euristic
df = resultsdf
df = df[['imp_bricks_avg', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Average of bricks'] = df.imp_bricks_avg

df1 = df.loc[df.resolution == 'low']
df2 = df.loc[df.resolution == 'medium']
df3 = df.loc[df.resolution == 'high']

#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
fig, axs = plt.subplots(1,3,constrained_layout=True )

sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
g1 = sns.boxplot(data=df1, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[0])
g2 = sns.boxplot(data=df2, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[1])
g3 = sns.boxplot(data=df3, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[2])
g1.set(xlabel=None)
g3.set(xlabel=None)
g1.set(ylabel="Average number of bricks after optimization")
g2.set(ylabel=None)
g3.set(ylabel=None)
axs[2].legend([],[], frameon=False)
axs[1].legend([],[], frameon=False)

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

# Tests on best statistics

In [336]:
bestcomponentsdf

,Unnamed: 0,model,resolution,execution_type,bricks_conn_comps,voxels_conn_comps,component_outside,imp_bricks_conn_comps,imp_voxels_conn_comps,imp_component_outside
0,0,chinese_lion.ply,low,random,"[512, 1]","[1853, 1]","[True, True]",[501],[1854],[True]
1,1,chinese_lion.ply,low,euristic,[415],[1854],[True],[415],[1854],[True]
2,2,chinese_lion.ply,medium,random,"[3917, 2, 2, 2, 2, 2, 1]","[14531, 4, 6, 4, 4, 2, 2]","[True, True, True, True, True, True, True]",[3789],[14553],[True]
3,3,chinese_lion.ply,medium,euristic,[2896],[14553],[True],[2896],[14553],[True]
4,4,chinese_lion.ply,high,random,"[33436, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,...","[127683, 3, 9, 2, 7, 5, 3, 2, 2, 1, 1, 2, 1, 2...","[True, True, True, True, True, True, True, Tru...",[32578],[127731],[True]
...,...,...,...,...,...,...,...,...,...,...
107,107,blockLP.ply,high,euristic,[23571],[128060],[True],[23571],[128060],[True]
108,108,lion_head.ply,low,random,"[480, 2, 1]","[1703, 3, 3]","[True, True, True]",[468],[1709],[True]
109,109,lion_head.ply,low,euristic,[387],[1709],[True],[387],[1709],[True]
110,110,lion_head.ply,medium,random,"[3710, 2, 2, 2, 1, 1]","[13945, 8, 3, 6, 1, 1]","[True, True, True, True, True, True]",[3617],[13964],[True]


In [5]:
df = bestcomponentsdf
df = df[['bricks_conn_comps', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Least number of bricks'] = df.bricks_conn_comps.apply(np.sum)

df1 = df.loc[df.resolution == 'low']
df2 = df.loc[df.resolution == 'medium']
df3 = df.loc[df.resolution == 'high']

#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
fig, axs = plt.subplots(1,3,constrained_layout=True )

sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
g1 = sns.boxplot(data=df1, x='Resolution', y='Least number of bricks', hue='Execution type',ax=axs[0])
g2 = sns.boxplot(data=df2, x='Resolution', y='Least number of bricks', hue='Execution type',ax=axs[1])
g3 = sns.boxplot(data=df3, x='Resolution', y='Least number of bricks', hue='Execution type',ax=axs[2])
g1.set(xlabel=None)
g3.set(xlabel=None)
g1.set(ylabel="Number of bricks in the best layouts")
g2.set(ylabel=None)
g3.set(ylabel=None)
axs[2].legend([],[], frameon=False)
axs[1].legend([],[], frameon=False)

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [9]:
# Plot bricks over resolution in random v euristic
df = bestcomponentsdf
df = df[['bricks_avg', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Average of bricks'] = df.bricks_avg

df1 = df.loc[df.resolution == 'low']
df2 = df.loc[df.resolution == 'medium']
df3 = df.loc[df.resolution == 'high']

#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
fig, axs = plt.subplots(1,3,constrained_layout=True )

sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Average of elapsed time', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
g1 = sns.boxplot(data=df1, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[0])
g2 = sns.boxplot(data=df2, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[1])
g3 = sns.boxplot(data=df3, x='Resolution', y='Average of bricks', hue='Execution type',ax=axs[2])
g1.set(xlabel=None)
g3.set(xlabel=None)
g1.set(ylabel="Average number of bricks")
g2.set(ylabel=None)
g3.set(ylabel=None)
axs[2].legend([],[], frameon=False)
axs[1].legend([],[], frameon=False)

plt.plot()

KeyError: "['bricks_avg'] not in index"

In [6]:
df = bestcomponentsdf
df = df[['voxels_conn_comps', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Voxels in secondary connected components'] = df.voxels_conn_comps.apply(lambda x: np.sum(x[1:]))
#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
plt.figure()
sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Voxels in connected components', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
#sns.swarmplot(data=df, x='Resolution', y='Voxels in secondary connected components', split=True, hue='Execution type',)
g1 = sns.boxplot(data=df, x='Resolution', y='Voxels in secondary connected components', hue='Execution type')

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [8]:
df = bestcomponentsdf
df = df[['voxels_conn_comps', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Voxels in secondary connected components'] = df.voxels_conn_comps.apply(lambda x: np.mean(x[1:]))
df['Voxels in secondary connected components'] = df['Voxels in secondary connected components'].fillna(0)

#df.resolution = df.resolution.apply(lambda x: ['low','medium', 'high'].index(x))
plt.figure()
sns.set_theme(style="whitegrid")
#sns.violinplot(data=df, x='Resolution', y='Voxels in connected components', split=True, hue='Execution type', scale="count");plt.legend(title="Execution type", loc='upper left')
#sns.swarmplot(data=df, x='Resolution', y='Voxels in secondary connected components', split=True, hue='Execution type',)
g1 = sns.boxplot(data=df, x='Resolution', y='Voxels in secondary connected components', hue='Execution type')

plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [8]:
df = bestcomponentsdf
df = df[['model', 'voxels_conn_comps', 'execution_type', 'resolution']]
df['Execution type'] = df['execution_type'].apply(lambda x: "Random" if x == "random" else "OP")
df['Resolution'] = df.resolution
df['Voxels in secondary connected components'] = df.voxels_conn_comps.apply(lambda x: np.sum(x[1:]))
df['Voxels in secondary connected components'] = df['Voxels in secondary connected components'].fillna(0)

df.loc[df['Voxels in secondary connected components'].idxmax()]


model                                             CoverSlipCleaningHolder2b.ply
voxels_conn_comps                           [20653, 60, 56, 64, 36, 18, 16, 16]
execution_type                                                         euristic
resolution                                                               medium
Execution type                                                               OP
Resolution                                                               medium
Voxels in secondary connected components                                  266.0
Name: 39, dtype: object

In [13]:
df = bestsdf
df['best_num_components'] = df.connected_components.apply(min)
df['percentage_success'] = df.connected_components.apply(lambda x: x.count(1)/5)
df['best_node_connectivity'] = df.apply(lambda row: row.node_connectivity[row.best_id],axis=1)
df['best_number_of_bricks'] = df.apply(lambda row: row.bricks[row.best_id],axis=1)
df['expected_time'] = df.apply(lambda row: np.sum(row.times)/(row.percentage_success if row.percentage_success != 0 else 1),axis=1)
df = df[['model','resolution', 'execution_type', 'best_num_components', 'percentage_success', 'best_number_of_bricks', 
         'best_node_connectivity', 'expected_time']]
group = df.groupby(["resolution","execution_type"], as_index=False)
table = group.mean()
table.to_csv("../out/table5.csv")
table


,resolution,execution_type,best_num_components,percentage_success,best_number_of_bricks,best_node_connectivity,expected_time
0,high,euristic,1.259259,0.674074,24926.925926,4.925926,1397.220075
1,high,random,9.370370,0.029630,33837.592593,3.918519,1636.646195
2,low,euristic,1.214286,0.635714,435.964286,2.957143,4.497617
3,low,random,2.571429,0.207143,527.928571,3.414286,5.709958
4,medium,euristic,1.481481,0.607407,2946.185185,3.592593,50.605357
5,medium,random,4.444444,0.103704,3973.703704,3.651852,45.745092


In [29]:
df = bestsdf
df['best_num_components'] = df.imp_connected_components.apply(min)
df['percentage_success'] = df.imp_connected_components.apply(lambda x: x.count(1)/5)
df['best_node_connectivity'] = df.apply(lambda row: row.imp_node_connectivity[row.imp_best_id],axis=1)
df['expected_time'] = df.apply(lambda row: np.sum(row.imp_times)/(row.percentage_success if row.percentage_success != 0 else 1),axis=1)
df['imp_avg_time'] = df.apply(lambda row: 0 if 1 in row.connected_components else np.mean(row.imp_times) ,axis=1)
df['iters'] = df.imp_iterations.apply(np.mean)
df = df[['model','resolution', 'execution_type', 'best_num_components', 'percentage_success', 'best_node_connectivity', 'imp_avg_time', 'iters']]
group = df.groupby(["resolution","execution_type"], as_index=False)
table = group.mean()
table.to_csv("../out/table6.csv")
table

,resolution,execution_type,best_num_components,percentage_success,best_node_connectivity,imp_avg_time,iters
0,high,euristic,1.035714,0.964286,4.928571,26.629199,1.092857
1,high,random,1.034483,0.965517,3.862069,293.231776,8.848276
2,low,euristic,1.000000,1.000000,2.944828,0.118295,0.351724
3,low,random,1.000000,1.000000,3.337931,0.751375,1.937931
4,medium,euristic,1.000000,0.986207,3.613793,3.483279,1.462069
5,medium,random,1.000000,1.000000,3.634483,12.056504,3.710345


In [43]:
df = bestsdf
df['best_num_components'] = df.imp_connected_components.apply(min)
df['percentage_success'] = df.imp_connected_components.apply(lambda x: x.count(1)/5)
df['best_node_connectivity'] = df.apply(lambda row: row.imp_node_connectivity[row.imp_best_id],axis=1)
df['expected_time'] = df.apply(lambda row: np.sum(row.imp_times)/(row.percentage_success if row.percentage_success != 0 else 1),axis=1)
df['imp_avg_time'] = df.apply(lambda row: 0 if 1 in row.connected_components else np.mean(row.imp_times) ,axis=1)
df['imp_avg_time2'] = df.apply(lambda row: np.mean(row.imp_times) ,axis=1)
df['iters'] = df.imp_iterations.apply(np.mean)
df1_truncated= resultsdf.tail(len(df))

best_data = df.loc[(df.resolution=='high') & (df.execution_type == 'random')]['imp_avg_time']
avg_data = df1_truncated.loc[(df1_truncated.resolution=='high') & (df1_truncated.execution_type == 'random')]['imp_time_avg']
best_data = df['imp_avg_time']
avg_data = df1_truncated['imp_time_avg']
print("a",best_data.mean())
print("b",avg_data.mean())
print(stats.ttest_rel(best_data, avg_data))

testdf = pd.DataFrame(df.resolution, df.execution_type, best_data, avg_data)
testdf

a 56.215105398266324
b 54.66852531309071
Ttest_relResult(statistic=0.08013633113197707, pvalue=0.9362219913997782)


imp_avg_time,0.478370,0.000000,16.736348,0.000000,599.488571,0.000000,0.940917,0.000000,11.106684,0.000000,...,0.000000,16.884301,4.066418,64.697839,0.000000,0.382997,0.000000,7.387544,0.000000,131.736075
execution_type,,,,,,,,,,,,,,,,,,,,,
random,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
euristic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
random,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
euristic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
random,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
random,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
euristic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
random,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df = bestsdf
df['best_num_components'] = df.imp_connected_components.apply(min)
df['percentage_success'] = df.imp_connected_components.apply(lambda x: x.count(1)/5)
df['best_node_connectivity'] = df.apply(lambda row: row.imp_node_connectivity[row.imp_best_id],axis=1)
df['expected_time'] = df.apply(lambda row: np.sum(row.imp_times)/(row.percentage_success if row.percentage_success != 0 else 1),axis=1)
df['imp_avg_time'] = df.apply(lambda row: 0 if 1 in row.connected_components else np.mean(row.imp_times) ,axis=1)
df['iters'] = df.imp_iterations.apply(np.mean)
df = df[['model','resolution', 'execution_type', 'best_num_components', 'percentage_success', 'best_node_connectivity', 'imp_avg_time', 'iters']]
group = df.groupby(["resolution","execution_type"], as_index=False)
table = group.mean()
#table.to_csv("../out/table7.csv")
table

,resolution,execution_type,best_num_components,percentage_success,best_node_connectivity,imp_avg_time,iters
0,high,euristic,1.035714,0.964286,4.928571,26.629199,1.092857
1,high,random,1.034483,0.965517,3.862069,293.231776,8.848276
2,low,euristic,1.000000,1.000000,2.944828,0.118295,0.351724
3,low,random,1.000000,1.000000,3.337931,0.751375,1.937931
4,medium,euristic,1.000000,0.986207,3.613793,3.483279,1.462069
5,medium,random,1.000000,1.000000,3.634483,12.056504,3.710345


In [64]:
bestcomponentsdf

,Unnamed: 0,model,resolution,execution_type,bricks_conn_comps,voxels_conn_comps,component_outside,imp_bricks_conn_comps,imp_voxels_conn_comps,imp_component_outside
0,0,chinese_lion.ply,low,random,"[512, 1]","[1853, 1]","[True, True]",[501],[1854],[True]
1,1,chinese_lion.ply,low,euristic,[415],[1854],[True],[415],[1854],[True]
2,2,chinese_lion.ply,medium,random,"[3917, 2, 2, 2, 2, 2, 1]","[14531, 4, 6, 4, 4, 2, 2]","[True, True, True, True, True, True, True]",[3789],[14553],[True]
3,3,chinese_lion.ply,medium,euristic,[2896],[14553],[True],[2896],[14553],[True]
4,4,chinese_lion.ply,high,random,"[33436, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,...","[127683, 3, 9, 2, 7, 5, 3, 2, 2, 1, 1, 2, 1, 2...","[True, True, True, True, True, True, True, Tru...",[32578],[127731],[True]
...,...,...,...,...,...,...,...,...,...,...
183,183,pegaso.ply,medium,euristic,[3432],[16387],[True],[3432],[16387],[True]
184,184,pegaso.ply,high,random,"[36050, 7, 5, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1,...","[136599, 16, 11, 6, 7, 5, 3, 2, 2, 2, 1, 4, 2,...","[True, True, True, True, True, True, True, Tru...",[35111],[136690],[True]
185,185,pegaso.ply,high,euristic,[26628],[136690],[True],[26628],[136690],[True]
186,186,dino0.ply,low,random,"[544, 2, 1, 1, 1]","[1890, 4, 3, 1, 1]","[True, True, True, True, True]",[537],[1899],[True]


In [8]:
diff.hist('perc')
from scipy.stats import norm

# x axis for normal plot
x_axis = np.arange(diff['perc'].min(), diff['perc'].max(), 0.001)

plt.plot(x_axis, norm.pdf(x_axis,diff['perc'].mean(),diff['perc'].std()))

diff['perc'].std()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.11134799683183054

In [9]:
# Plot bricks over voxels in random v euristic
x_axis_r = df.loc[(df['execution_type'] == 'random' )]['voxels']
y_axis_r = df.loc[(df['execution_type'] == 'random' )]['bricks_avg']

x_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['voxels']
y_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['bricks_avg']

x_axis_ri = df.loc[(df['execution_type'] == 'random' )]['voxels']
y_axis_ri = df.loc[(df['execution_type'] == 'random' )]['imp_bricks_avg']

x_axis_ei = df.loc[(df['execution_type'] == 'euristic' )]['voxels']
y_axis_ei = df.loc[(df['execution_type'] == 'euristic' )]['imp_bricks_avg']

plt.figure()
plt.plot(x_axis_r, y_axis_r, 'ob')
plt.plot(x_axis_e, y_axis_e, 'or')
#plt.plot(x_axis_ri, y_axis_ri, 'om')
#plt.plot(x_axis_ei, y_axis_ei, 'og')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Plot time over voxels in random v euristic
x_axis_r = df.loc[(df['execution_type'] == 'random' )]['voxels']
y_axis_r = df.loc[(df['execution_type'] == 'random' )]['time_avg']

x_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['voxels']
y_axis_e = df.loc[(df['execution_type'] == 'euristic' )]['time_avg']

x_axis_ri = df.loc[(df['execution_type'] == 'random' )]['voxels']
y_axis_ri = df.loc[(df['execution_type'] == 'random' )]['imp_time_avg']

x_axis_ei = df.loc[(df['execution_type'] == 'euristic' )]['voxels']
y_axis_ei = df.loc[(df['execution_type'] == 'euristic' )]['imp_time_avg']

plt.figure()
plt.plot(x_axis_r, y_axis_r, 'ob')
plt.plot(x_axis_e, y_axis_e, 'or')
#plt.plot(x_axis_ri, y_axis_ri+y_axis_r, 'om')
#plt.plot(x_axis_ei, y_axis_ei+y_axis_e, 'og')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
# metrics on voxelizations
single_lego_sizes = np.array([8,8,9.6])
num_reapeat = 10
num_connectivity_pair = 10
pathlist = Path("../meshes/dataset/").glob("*.ply")
modeldf = pd.DataFrame(columns=["model", "resolution", "volume", "voxels", "overhangs", "unsupported", "outside", "surface"])
for path in pathlist:
    mesh_name = str(path)

    mesh = trimesh.load(mesh_name)
    rot_angle = lego.optimal_rotation_angle(mesh)
    mesh = utils.rotate_mesh(mesh, -rot_angle, [0,0,1])
    print(mesh_name, mesh.is_watertight)
    
    #for lego_base_size in [15, 31, 63]: #a +1 is added implicitly to base size
    for target_volume, resolution in [(1e6*.7,'low'), (1e7*.7,'medium'), (1e8*.7,'high')]: #a +1 is added implicitly to base size
        #mesh = utils.fit_mesh_to_plate(mesh, lego_base_size, single_lego_sizes[0])
        mesh = utils.fit_mesh_to_volume(mesh, target_volume, single_lego_sizes[0])
        trimesh.exchange.export.export_mesh(mesh, f"../out/meshes/{path.parts[-1]}_{resolution}.ply")
        trimesh.exchange.export.export_mesh(mesh.voxelized(pitch=single_lego_sizes).as_boxes(), f"../out/meshes/{path.parts[-1]}_{resolution}_blocks.ply")
        
        voxels = mesh.voxelized(pitch=single_lego_sizes).fill().matrix
        outside = mesh.voxelized(pitch=single_lego_sizes).matrix
        np.save(f"../out/ndarrays/{path.parts[-1]}_{resolution}_fill.npy", voxels, allow_pickle=False)
        np.save(f"../out/ndarrays/{path.parts[-1]}_{resolution}_out.npy", outside, allow_pickle=False)
        
        unsupported = 0
        for iz in range(1, voxels.shape[2]):
            for ix in range(voxels.shape[0]):
                for iy in range(voxels.shape[1]):
                    if voxels[ix,iy,iz] != 0 and voxels[ix,iy,iz-1] == 0:
                        unsupported += 1
        
        overhangs = 0
        for ix in range(voxels.shape[0]):
            for iy in range(voxels.shape[1]):
                overhang = False
                for iz in range(1, voxels.shape[2]):
                    if voxels[ix,iy,iz] == 0:
                        overhang = True
                    elif overhang is True and voxels[ix,iy,iz] != 0:
                        overhangs += 1
        
        surface = 0
        for ix in range(voxels.shape[0]):
            for iy in range(voxels.shape[1]):
                for iz in range(1, voxels.shape[2]):
                    if voxels[ix,iy,iz] != 0:
                        if ix == 0 or iy == 0 or iz == 0 or ix == voxels.shape[0]-1 or iy == voxels.shape[1]-1 or iz == voxels.shape[2]-1:
                            surface += 1
                        elif voxels[ix-1,iy,iz] == 0 or voxels[ix+1,iy,iz] or voxels[ix,iy-1,iz] == 0 or voxels[ix,iy+1,iz] == 0 or voxels[ix,iy,iz-1] == 0 or voxels[ix,iy,iz+1] == 0:
                            surface +=1

        modeldf.loc[modeldf.shape[0]] = [path.parts[-1], resolution, mesh.volume, int(voxels.sum()), overhangs, unsupported, outside.sum(), surface]
        
modeldf = modeldf.apply(pd.to_numeric, errors='ignore')
modeldf.dtypes

../meshes/dataset/chinese_lion.ply True
../meshes/dataset/buste.ply True
../meshes/dataset/pierrot100k.ply True
../meshes/dataset/elephant.ply True
../meshes/dataset/dragonstand_recon100K.ply True
../meshes/dataset/david.ply True
../meshes/dataset/CoverSlipCleaningHolder2b.ply True
../meshes/dataset/rabbit5k.ply True
../meshes/dataset/buddha.ply True
../meshes/dataset/bamboo_pen.ply True
../meshes/dataset/bimba.ply True
../meshes/dataset/dancing_children100K.ply True
../meshes/dataset/indorelax.ply True
../meshes/dataset/ramses.ply True
../meshes/dataset/camel-tri.ply True
../meshes/dataset/armchair.ply True
../meshes/dataset/cactus.ply True
../meshes/dataset/blockLP.ply True
../meshes/dataset/lion_head.ply True
../meshes/dataset/3_holes.ply True
../meshes/dataset/kitten.ply True
../meshes/dataset/greek_sculpture.ply True
../meshes/dataset/teapot.ply True
../meshes/dataset/armadillo.ply True
../meshes/dataset/botijo-tri.ply True
../meshes/dataset/gargoyle2_100K.ply True
../meshes/datas

model           object
resolution      object
volume         float64
voxels           int64
overhangs        int64
unsupported      int64
outside          int64
surface          int64
dtype: object

In [296]:
#modeldf.hist('voxels')
df2 = modeldf
df2["ratio"] = modeldf.outside/modeldf.voxels
df2.loc[(df2.ratio == 1)]

,model,resolution,voxels,overhangs,unsupported,outside,surface,ratio
27,bamboo_pen.ply,15,852,50,20,852,613,1.0
